In [2]:
import gpc
import itertools
import math
import re

In [3]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei

In [4]:
def isEipal(seq, i):
    "kontroluje jestli řetězec je pseudopalindrom"
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == str(i):
            return True
        else:
            return False
    for x in range(0, math.ceil(l/2)):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [5]:
def makeEipalClosure (seq, i):
    "udělá z řetězce pseudopalindromický uzávěr"
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        j = j+1
    #print("    {0} nejdelší pseudopalindromický sufix : {1}".format(seq,seq[j:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [6]:
def make012Word(delta, theta, steps, seed = ""):
    "vytvoří slovo pomocí řídící posloupnosti a posloupnosti uzávěrů"
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        #print("w{0} = {1}".format(step+1,w))
    return(w)

## Zakázané prefixy

In [7]:
def normalize(delta, theta):
    """Returns the normalized directive bi-sequence giving the same GPS word
    as (delta, theta)"""
    biseq = makeBiseq(delta, theta)
    bad_matches = find_bad_factors()
    
    
    

In [23]:
print(re.search('01$', '010101'))

<_sre.SRE_Match object; span=(4, 6), match='01'>


is012Normalized("0100001","R012RR2", 7)

In [ ]:
isEipal("012", 1)

In [ ]:
a = [0,1, "ahoj"]
a = [a, "dalsi"]